## Performing a mocserver query on a simple cone region

In [17]:
from astropy import coordinates
from regions import CircleSkyRegion

# mocserver object derived from astroquery.query.BaseQuery
from cds.core import cds
from cds.dataset import Dataset
from cds.constraints import Constraints
from cds.spatial_constraints import Cone
from cds.output_format import OutputFormat

from pprint import pprint

A cone search region is defined using the CircleSkyRegion module from the regions package.
Here, a cone is defined by a location (dec, ra) expressed in degree and a radius.

In [18]:
center = coordinates.SkyCoord(10.8, 32.2, unit="deg")
radius = coordinates.Angle(1.5, 'deg')
circle_sky_region = CircleSkyRegion(center, radius)

This above cone region can be seen as a spatial constraint for the mocserver. The aim of the mocserver relies on returning all the datasets which contain at least one source (i.e. spatial object) lying in the previously defined cone region.

A Constraints object allows us to specify the constraints that will be sent to the mocserver so that it returns all the matching datasets. For the purpose of this tutorial, we will only bind a Cone spatial constraint to our query but keep in mind that it is also possible to associate to the Constraints object a constraint on the dataset properties. It is even possible to associate both of them (i.e. a spatial constraint and a properties constraint) as we will see in a future tutorial.

In [19]:
cds_constraints = Constraints(sc=Cone(circle_sky_region, intersect='overlaps'))

Here we have created a Constraints object and we have bound to it a Cone spatial constraint. The argument `intersect` specifies that the datasets need to intersect the cone region so that they match the mocserver query. Other possible `intersect` argument values are `covers` and `enclosed`.

Now it is time to perform the query. We call the query_region method from the mocserver object that we imported and pass it the mocserver constraints object with an OutputFormat object which specify what we want to retrieve. In the code below, we have chosen to retrieve all the properties from the datasets, but keep in mind that it is possible, as in the MocServer http://alasky.unistra.fr/MocServer/query, to get only some specific fields from the matching datasets (`ID`, `dataproduct_type` and `moc_sky_fraction` are typical example of fields/properties bound to a dataset).

Finally, we pass a `max_rec` parameter to the OutputFormat constructor which limits the number of matching datasets returned (otherwise we would have a plenty of datasets for this cone spatial constraint).

In [20]:
datasets_d = cds.query_region(cds_constraints,
                                  OutputFormat(format=OutputFormat.Type.record,
                                               max_rec=5))

pprint(datasets_d)

Final Request payload before requesting to alasky
{'DEC': '32.2',
 'MAXREC': '5',
 'RA': '10.8',
 'SR': '1.5',
 'casesensitive': 'true',
 'fmt': 'json',
 'get': 'record',
 'intersect': 'overlaps'}
{'CDS/B/assocdata/obscore': <cds.dataset.Dataset object at 0x7f378a979b00>,
 'CDS/B/cb/lmxbdata': <cds.dataset.Dataset object at 0x7f378a979ba8>,
 'CDS/B/cfht/cfht': <cds.dataset.Dataset object at 0x7f378a979c18>,
 'CDS/B/cfht/obscore': <cds.dataset.Dataset object at 0x7f378a979c88>,
 'CDS/B/chandra/chandra': <cds.dataset.Dataset object at 0x7f378a979cf8>}


We got a dictionary of mocserver.dataset.Dataset objects indexed by their IDs. To get the properties of one dataset, say `CDS/B/cb/lmxbdata`, just do the following :

In [21]:
pprint(datasets_d['CDS/B/cb/lmxbdata'].properties)

{'ID': 'CDS/B/cb/lmxbdata',
 'TIMESTAMP': 1521555687000.0,
 'bib_reference': '2003A&A...404..301R',
 'cs_service_url': 'http://vizier.u-strasbg.fr/viz-bin/votable/-A?-source=B%2Fcb%2Flmxbdata&',
 'data_ucd': ['src.class',
              'phys.luminosity;arith.ratio',
              'time.period',
              'phot.mag;em.opt.V',
              'src.orbital.inclination',
              'pos.eq.dec',
              'phys.mass',
              'pos.eq.ra',
              'phys.mass;arith.ratio',
              'meta.id',
              'src.spType'],
 'dataproduct_type': 'catalog',
 'moc_access_url': 'http://alasky.unistra.fr/footprints/tables/vizier/B_cb_lmxbdata/MOC?nside=2048',
 'moc_order': 11.0,
 'moc_sky_fraction': 2.066e-06,
 'nb_rows': 108.0,
 'obs_astronomy_kw': ['Novae', 'Binaries:cataclysmic'],
 'obs_collection': 'CV',
 'obs_collection_label': ['CV', 'CB'],
 'obs_copyright_url': 'http://cdsarc.u-strasbg.fr/viz-bin/Cat?B%2Fcb',
 'obs_description': 'Catalogue of Low-Mass X-Ray Binaries'

It is also possible to get only the datasets ``ID``s, the ``number``
of matching datasets or just the ``moc`` resulting from the union of all
the mocs of the matching datasets (mocpy object). (See the OutputFormat definition class and its ``format`` type).

In [22]:
moc = cds.query_region(cds_constraints,
                                  OutputFormat(format=OutputFormat.Type.moc,
                                               moc_order=14))
pprint(moc)

Final Request payload before requesting to alasky
{'DEC': '32.2',
 'RA': '10.8',
 'SR': '1.5',
 'casesensitive': 'true',
 'fmt': 'json',
 'get': 'moc',
 'intersect': 'overlaps',
 'order': 14}


The result of a mocserver query depends on the OutputFormat object we have passed to the query_region method.
If we query for the ``ID``s of the datasets then we get a python list of all the  ``ID``s matching the condition.
If a user query for the ``number`` of datasets, he gets an int.
If you want all the record of the matching datasets (the ``ID``s plus its properties/fields) you get
as we have seen above, a dictionary indexed by the ``ID``s of the datasets and as each value a Dataset object.
This Dataset object allows you to query a specific service (if available for this dataset) such as a tap service, a cone search service...

The service then returns a VOTable containing all the matching sources of the dataset you have submitted a query. We can see that the dataset `CDS/B/cb/lmxbdata` contains the field `cs_service_url` referring the url of the cone search service for this dataset. To query this service, we call the following command, specifying we want to query the Simple Cone Search service with a cone of center `pos` and of radius `radius`.

In [23]:
votable = datasets_d['CDS/B/cb/lmxbdata'].search(Dataset.ServiceType.cs, pos=(10.8, 32.2), radius=1.5)
votable

<Table masked=True length=1>
_RAJ2000 _DEJ2000    _r      Name   ...   mag1   Orb.Per  SpType2  Refs 
  deg      deg      deg             ...   mag       d                   
float64  float64  float64  bytes12  ... float32  float64   bytes7 bytes4
-------- -------- ------- --------- ... ------- --------- ------- ------
 11.2033  33.0092 0.87761 0042+3244 ...    18.8 11.600000           Refs

The returned VOTable shows us why the dataset has been returned by the mocserver : there is a source from that dataset which lies into the cone defined as the spatial constraint for the mocserver query.